In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd  

2024-11-16 11:41:41.052695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 11:41:41.401499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-16 11:41:41.401519: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-16 11:41:42.615199: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
base_dir = r'/home/prashantb/Documents/Prashant/Thesis/FinalData'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [3]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of training data used as validation data
)

In [4]:
# Data generator for training data
train_generator = train_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Set as training data
)

Found 13964 images belonging to 2 classes.


In [5]:
# Data generator for validation data
validation_generator = train_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/train',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    subset='validation'  # Set as validation data
)

Found 3490 images belonging to 2 classes.


In [6]:
# Data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/test',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)

Found 2096 images belonging to 2 classes.


In [16]:
from tensorflow.keras.callbacks import CSVLogger


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
import tensorflow as tf

# Define the complex CNN model
cnn_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    GlobalAveragePooling2D(),

    Dense(2048, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

# Compile the CNN model
cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [15]:
import pandas as pd  
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

log_file_path = 'cnn_training_log.csv'

# Create CSVLogger callback
csv_logger = CSVLogger(log_file_path, append=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [20]:
import time
import logging
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [21]:
start_time = time.time()

# Train the CNN model
history_cnn = cnn_model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, csv_logger]
)

# End timing
end_time = time.time()
execution_time_cnn = end_time - start_time

# Log training and evaluation results
logging.info(f'CNN Model - Training History: {history_cnn.history}')
logging.info(f'Execution Time for CNN Model: {execution_time_cnn} seconds')

# Evaluate the model on the validation data
y_true = validation_generator.classes
y_pred = (cnn_model.predict(validation_generator) > 0.5).astype("int32")

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Log metrics
logging.info(f'CNN Model - Accuracy: {accuracy:.4f}')
logging.info(f'CNN Model - Precision: {precision:.4f}')
logging.info(f'CNN Model - Recall: {recall:.4f}')
logging.info(f'CNN Model - F1-score: {f1:.4f}')

# Output results to the console
print(f"CNN Model - Accuracy: {accuracy:.4f}")
print(f"CNN Model - Precision: {precision:.4f}")
print(f"CNN Model - Recall: {recall:.4f}")
print(f"CNN Model - F1-score: {f1:.4f}")
print(f"CNN Model - Execution Time: {execution_time_cnn} seconds")

# Log the best achieved metric
best_accuracy = max(history_cnn.history['val_accuracy'])
best_precision = max(history_cnn.history['val_precision']) if 'val_precision' in history_cnn.history else precision
best_recall = max(history_cnn.history['val_recall']) if 'val_recall' in history_cnn.history else recall
best_f1 = max(history_cnn.history['val_f1_score']) if 'val_f1_score' in history_cnn.history else f1

logging.info(f'Best CNN Model - Accuracy: {best_accuracy:.4f}')
logging.info(f'Best CNN Model - Precision: {best_precision:.4f}')
logging.info(f'Best CNN Model - Recall: {best_recall:.4f}')
logging.info(f'Best CNN Model - F1-score: {best_f1:.4f}')

Epoch 1/25
437/437 [==============================] - 1247s 3s/step - loss: 1.9961 - accuracy: 0.7520 - val_loss: 4.1895 - val_accuracy: 0.7146 - lr: 1.0000e-04
Epoch 2/25
437/437 [==============================] - 1167s 3s/step - loss: 1.7554 - accuracy: 0.7835 - val_loss: 1.8143 - val_accuracy: 0.7178 - lr: 1.0000e-04
Epoch 3/25
437/437 [==============================] - 1166s 3s/step - loss: 1.5261 - accuracy: 0.8053 - val_loss: 1.5377 - val_accuracy: 0.7607 - lr: 1.0000e-04
Epoch 4/25
437/437 [==============================] - 1188s 3s/step - loss: 1.3360 - accuracy: 0.8037 - val_loss: 1.2356 - val_accuracy: 0.8590 - lr: 1.0000e-04
Epoch 5/25
437/437 [==============================] - 1230s 3s/step - loss: 1.1563 - accuracy: 0.8177 - val_loss: 1.1260 - val_accuracy: 0.7877 - lr: 1.0000e-04
Epoch 6/25
437/437 [==============================] - 1233s 3s/step - loss: 1.0191 - accuracy: 0.8212 - val_loss: 0.8179 - val_accuracy: 0.9034 - lr: 1.0000e-04
Epoch 7/25
437/437 [==============